# Tool-calling Agent

The agent code implements [MLflow's ChatAgent](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) interface, a Databricks-recommended open-source standard that simplifies authoring multi-turn conversational agents, and is fully compatible with Mosaic AI agent framework functionality.

**_NOTE:_** This notebook uses LangChain, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or pure Python agents written with the OpenAI SDK.

## Install Dependencies

In [1]:
%pip install uv

/Users/scott.mckean/Repos/hydrate_prediction/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%sh uv pip install .

UsageError: Line magic function `%sh` not found (But cell magic `%%sh` exists, did you mean that instead?).


## Tools

We could use something like Genie to talk with out data, but when the structures are simple, it makes sense to use a more lightweight approach. This is a simple example of using a SQL query as a tool.

In [ ]:
from hydrate.utils import DotConfig
config = DotConfig('config.yaml')

In [ ]:
spark.sql("""
    CREATE OR REPLACE FUNCTION shm.3w.filter_well_data(
        p_well_number INT COMMENT 'well number',
        p_num_minutes INT COMMENT 'number of minutes'
    )
    RETURNS TABLE (
        `T-JUS-CKP` DOUBLE,
        `T-TPT` DOUBLE,
        `P-TPT` DOUBLE,
        `P-MON-CKP` DOUBLE,
        `P-PDG` DOUBLE,
        timestamp TIMESTAMP
    )
    RETURN
    SELECT
        `T-JUS-CKP`,
        `T-TPT`,
        `P-TPT`,
        `P-MON-CKP`,
        `P-PDG`,
        timestamp
    FROM (
    SELECT
        *,
        MAX(timestamp) OVER (PARTITION BY well_number) AS max_ts
    FROM shm.3w.well_data
    )
    WHERE well_number = p_well_number
    AND timestamp >= date_add(MINUTE, -p_num_minutes, max_ts)
""")

## Agent

Let's test the agent. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

In [3]:
from agent import AGENT
AGENT.predict({"messages": [{"role": "user", "content": "Hello!"}]})

ModuleNotFoundError: No module named 'databricks_langchain'

In [ ]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "What is 5+5 in python"}]}
):
    print(event, "-----------\n")


## Log the agent as an MLflow model

Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/unstructured-retrieval-tools) or leverages [external functions](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/external-connection-tools), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).


In [ ]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from agent import LLM_ENDPOINT_NAME, tools
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        # TODO: If the UC function includes dependencies like external connection or vector search, please include them manually.
        # See the TODO in the markdown above for more information.
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What are the predictions for well 15 for the next minute?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
    )


## Evaluate the Agent

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

Evaluate your agent with one of our [predefined LLM scorers](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/predefined-judge-scorers), or try adding [custom metrics](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/custom-scorers).


In [ ]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness
from databricks import agents

eval_dataset = [
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "What are the predictions for well 15 for the next minute?"
                }
            ]
        },
        "expected_response": None
    }
]

eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda messages: AGENT.predict({"messages": messages}),
    scorers=[RelevanceToQuery(), Safety()], # add more scorers here if they're applicable
)

# Review the evaluation results in the MLfLow UI (see console output)


## Perform pre-deployment validation of the agent

Before registering and deploying the agent, we perform pre-deployment checks via the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See [documentation](https://learn.microsoft.com/azure/databricks/machine-learning/model-serving/model-serving-debug#validate-inputs) for details


In [ ]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)


## Register and Deploy the Agent

Once we register the model to Unity Catalog, serving it becomes quite simple. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/deploy-agent) for details

In [ ]:
mlflow.set_registry_uri("databricks-uc")

UC_MODEL_NAME = f"{config.catalog}.{config.schema}.{config.agentify.agent_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)


In [ ]:

agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})
